Tracking Token Usage

In [8]:
import os
from dotenv import load_dotenv

# from pathlib import Path
# dotenv_path = Path('path/to/.env')
# load_dotenv(dotenv_path=dotenv_path)

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ACTIVELOOP_TOKEN = os.getenv('ACTIVELOOP_TOKEN')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

llm = OpenAI(model_name="text-davinci-003", n=2, best_of=2)

with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    print(cb)

Tokens Used: 45
	Prompt Tokens: 4
	Completion Tokens: 41
Successful Requests: 1
Total Cost (USD): $0.0009000000000000001


### Few-shot learning

Few-shot learning is a remarkable ability that allows LLMs to learn and generalize from limited examples. Prompts serve as the input to these models and play a crucial role in achieving this feature. With LangChain, examples can be hard-coded, but dynamically selecting them often proves more powerful, enabling LLMs to adapt and tackle tasks with minimal training data swiftly.

This approach involves using the **FewShotPromptTemplateCopy** class, which takes in a **PromptTemplate** and a list of a few shot examples. The class formats the prompt template with a few shot examples, which helps the language model generate a better response. We can streamline this process by utilizing LangChain's FewShotPromptTemplate to structure the approach:

In [4]:
from langchain import PromptTemplate
from langchain import FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)

After creating a template, we pass the example and user query, and we get the results

In [7]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain

chain = LLMChain(llm=llm, prompt=few_shot_prompt_template)
chain.run("What's the meaning of life?")

' The meaning of life is to find your purpose, be true to yourself, and make an impact on the world!'

### Emergent abilities, Scaling laws, and hallucinations

Another aspect of LLMs is their **emergent abilities**, which arise as a result of extensive pre-training on vast datasets. These capabilities are not explicitly programmed but emerge as the model discerns patterns within the data. LangChain models capitalize on these emergent abilities by working with various types of models, such as chat models and text embedding models. This allows LLMs to perform diverse tasks, from answering questions to generating text and offering recommendations.

Lastly, **scaling laws** describe the relationship between model size, training data, and performance. Generally, as the model size and training data volume increase, so does the model's performance. However, this improvement is subject to diminishing returns and may not follow a linear pattern. It is essential to weigh the trade-off between model size, training data, performance, and resources spent on training when selecting and fine-tuning LLMs for specific tasks.

While Large Language Models boast remarkable capabilities but are not without shortcomings, one notable limitation is the **occurrence of hallucinations**, in which these models produce text that appears plausible on the surface but is actually factually incorrect or unrelated to the given input. Additionally, LLMs may **exhibit biases** originating from their training data, resulting in outputs that can perpetuate stereotypes or generate undesired outcomes.

### Examples with Easy Prompts: Text Summarization, Text Translation, and Question Answering

#### Creating a Question-Answering Prompt Template

Let's create a simple question-answering prompt template using LangChain

In [9]:
from langchain import PromptTemplate

template = """
Question: {question}
Answer: 
"""

prompt = PromptTemplate(
        input_variables=['question'],
        template=template
       
)

# user question
question = "What is the capital city of France?"

Next, we will use the Hugging Face model **google/flan-t5-large** to answer the question. The **HuggingfaceHub** class will connect to Hugging Face’s inference API and load the specified model.

In [14]:
from langchain import HuggingFaceHub, LLMChain

# initialize Hub LLM
hub_llm = HuggingFaceHub(
    repo_id='google/flan-t5-large',
    model_kwargs={'temperature':0}
)

# create prompt template > LLM chain
llm_chain = LLMChain(
    prompt=prompt,
    llm=hub_llm
)

# ask the user question about the capital of France
print(llm_chain.run(question))

paris


#### Asking Multiple Questions

To ask multiple questions, we can either iterate through all questions one at a time or place all questions into a single prompt for more advanced LLMs. Let's start with the first approach:

In [11]:
qa = [
    {'question': "What is the capital city of France?"},
    {'question': "What is the largest mammal on Earth?"},
    {'question': "Which gas is most abundant in Earth's atmosphere?"},
    {'question': "What color is a ripe banana?"}
]
res = llm_chain.generate(qa)
print(res)

generations=[[Generation(text='paris', generation_info=None)], [Generation(text='giraffe', generation_info=None)], [Generation(text='nitrogen', generation_info=None)], [Generation(text='yellow', generation_info=None)]] llm_output=None run=RunInfo(run_id=UUID('ef616d18-ad75-412e-aaac-d051f9641656'))


We can modify our prompt template to include multiple questions to implement a second approach. The language model will understand that we have multiple questions and answer them sequentially. This method performs best on more capable models.

In [16]:
multi_template = """Answer the following questions one at a time.

Questions:
{questions}

Answers:
"""

long_prompt = PromptTemplate(template=multi_template, input_variables=["questions"])

llm_chain = LLMChain(
    prompt=long_prompt,
    llm=llm
)

qs_str = (
    "What is the capital city of France?\n" +
    "What is the largest mammal on Earth?\n" +
    "Which gas is most abundant in Earth's atmosphere?\n" +
	"What color is a ripe banana?\n"
)
llm_chain.run(qs_str)

'Paris\nBlue Whale\nNitrogen\nYellow'

#### Text Summarization

Using LangChain, we can create a chain for text summarization. First, we need to set up the necessary imports and an instance of the OpenAI language model:

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

Next, we define a prompt template for summarization:

In [18]:
summarization_template = "Summarize the following text to one sentence: {text}"
summarization_prompt = PromptTemplate(input_variables=["text"], template=summarization_template)
summarization_chain = LLMChain(llm=llm, prompt=summarization_prompt)

To use the summarization chain, simply call the **predict** method with the text to be summarized:

In [20]:
text = """
LangChain provides many modules that can be used to build language model applications. Modules can be combined to create more complex applications, or be used individually for simple applications. 
The most basic building block of LangChain is calling an LLM on some input. Let’s walk through a simple example of how to do this. For this purpose, 
let’s pretend we are building a service that generates a company name based on what the company makes.
"""
summarized_text = summarization_chain.predict(text=text)
summarized_text

'LangChain offers various modules for building language model applications, which can be used individually or combined to create more complex applications, with the most basic building block being calling an LLM on input, as demonstrated in a simple example of generating a company name based on its product.'

#### Text Translation

It is one of the great attributes of Large Language models that enables them to perform multiple tasks just by changing the prompt. We use the same llmCopy variable as defined before. However, pass a different prompt that asks for translating the query from a **source_language** to the **target_language**.

In [22]:
translation_template = "Translate the following text from {source_language} to {target_language}: {text}"

translation_prompt = PromptTemplate(
    input_variables=["source_language", "target_language", "text"], 
    template=translation_template
)

translation_chain = LLMChain(
    llm=llm, 
    prompt=translation_prompt
)

To use the translation chain, call the **predict** method with the source language, target language, and text to be translated:

In [24]:
source_language = "English"
target_language = "French"
text = "Your text here"
translated_text = translation_chain.predict(source_language=source_language, target_language=target_language, text=text)
translated_text

'Votre texte ici'